# Exploring the location data of disaster tweets

In [1]:
import pandas as pd
train = pd.read_csv('data/train.csv')
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [2]:
locations = train.loc[~train['location'].isnull(), ['location','target']]
locations.head()

,location,target
31,Birmingham,1
32,Est. September 2012 - Bristol,0
33,AFRICA,1
34,"Philadelphia, PA",0
35,"London, UK",0


# Use the Nominatim geocoder to find the coordinates of the locations

In [3]:
from geopandas.tools import geocode

### Encode a single location

In [4]:
locations['location'].iloc[0]

'Birmingham'

In [5]:
from geopy import Nominatim
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode(locations['location'].iloc[0])

In [6]:
location

Location(Birmingham, West Midlands Combined Authority, England, United Kingdom, (52.4796992, -1.9026911, 0.0))

In [7]:
print(location.longitude, location.latitude, location.altitude)

-1.9026911 52.4796992 0.0


In [8]:
location.point

Point(52.4796992, -1.9026911, 0.0)

### Encode all

In [38]:
import os.path
coded_locations = pd.Series(dtype='object')
offset = 0
CODED_LOCATIONS_PICKLE = 'coded_locations.pickle'

In [74]:
import os
if os.path.exists(CODED_LOCATIONS_PICKLE):
    coded_locations = pd.read_pickle(CODED_LOCATIONS_PICKLE)
    offset = len(coded_locations)

In [75]:
from geopy.extra.rate_limiter import RateLimiter
from geopy import Nominatim

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
locator = Nominatim(user_agent="myGeocoder")

CHUNK_SIZE = 60
for i in range(offset, len(locations), CHUNK_SIZE):
    print(i)
    with open("offset.txt","w+") as f:
        f.write(str(i))
    end = i + CHUNK_SIZE
    if end>len(locations):
        end = len(locations)    
    coded_locations = coded_locations.append(locations.iloc[i:end]['location'].apply(geocode))
    coded_locations.to_pickle(CODED_LOCATIONS_PICKLE)

In [76]:
len(coded_locations)

5080

In [77]:
len(locations)

5080

## Looking at missing geocodes

In [78]:
print(f"Missing: {round(100*sum(coded_locations.isnull())/len(coded_locations))}%")

Missing: 19%


In [87]:
points = coded_locations.apply(lambda loc: tuple(loc.point) if loc else None)
points
train[['latitude', 'longitude', 'altitude']] = pd.DataFrame(points.tolist(), index=points.index)

In [88]:
coded_locations[coded_locations.isnull()]="unknown"

In [89]:
train['coded_locations'] = coded_locations

In [90]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               7613 non-null   int64  
 1   keyword          7552 non-null   object 
 2   location         5080 non-null   object 
 3   text             7613 non-null   object 
 4   target           7613 non-null   int64  
 5   coded_locations  5080 non-null   object 
 6   geocoded         7613 non-null   bool   
 7   latitude         4116 non-null   float64
 8   longitude        4116 non-null   float64
 9   altitude         4116 non-null   float64
dtypes: bool(1), float64(3), int64(2), object(4)
memory usage: 542.8+ KB


In [92]:
train[train['coded_locations']=='unknown'].head(10)

,id,keyword,location,text,target,coded_locations,geocoded,latitude,longitude,altitude
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,unknown,False,NaN,NaN,NaN
40,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,0,unknown,False,NaN,NaN,NaN
48,68,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,0,unknown,False,NaN,NaN,NaN
59,83,ablaze,"Edmonton, Alberta - Treaty 6",How the West was burned: Thousands of wildfire...,1,unknown,False,NaN,NaN,NaN
61,86,ablaze,Inang Pamantasan,Progressive greetings!\n\nIn about a month stu...,0,unknown,False,NaN,NaN,NaN
62,89,ablaze,Twitter Lockout in progress,Rene Ablaze &amp; Jacinta - Secret 2k13 (Falle...,0,unknown,False,NaN,NaN,NaN
67,96,accident,CLVLND,'I can't have kids cuz I got in a bicycle acci...,0,unknown,False,NaN,NaN,NaN
81,118,accident,Your Sister's Bedroom,I was in a horrible car accident this past Sun...,1,unknown,False,NaN,NaN,NaN
102,146,aftershock,Instagram - @heyimginog,@afterShock_DeLo scuf ps live and the game... cya,0,unknown,False,NaN,NaN,NaN
108,158,aftershock,Instagram - @heyimginog,@afterShock_DeLo im speaking from someone that...,0,unknown,False,NaN,NaN,NaN


Missing geocodes seem to be mostly non-existing locations in the world. Is that a good feature?

In [93]:
train['geocoded'] = (train['coded_locations']!="unknown") & (~train['coded_locations'].isnull())

In [94]:
locations = train[~train['location'].isnull()]

In [95]:
unknown_locations = locations[locations['coded_locations']=="unknown"]
print(f"Disasters at unknown locations: {round(100*unknown_locations['target'].mean())}%") 

Disasters at unknown locations: 38%


In [96]:
known_locations = locations[locations['coded_locations']!="unknown"]
print(f"Disasters at known locations: {round(100*known_locations['target'].mean())}%") 

Disasters at known locations: 44%


Not such a super distinction, but might help.

## Looking at the location of the geocodes found

In [97]:
number_coded = sum((train['coded_locations'].isnull())&(train['coded_locations']!="unknown"))
print(f"Percent geocoded in trainingset: {round(100*number_coded/len(train))}%")

Percent geocoded in trainingset: 33%


### Disaster tweets

In [150]:
from folium.plugins import HeatMap
locs = known_locations[known_locations['target']==1]
hm_wide = HeatMap( list(zip(locs.latitude.values, locs.longitude.values, [1]*len(locs))),
                   min_opacity=0.2,
                   max_val=2,
                   radius=7, blur=12, 
                   max_zoom=1, 
                 )
hmap = folium.Map(location=[0,0],zoom_start=2)
hmap.add_child(hm_wide)
hmap

### Non disaster tweets

In [149]:
from folium.plugins import HeatMap
locs = known_locations[known_locations['target']==0]
hm_wide = HeatMap( list(zip(locs.latitude.values, locs.longitude.values, [1]*len(locs))),
                   min_opacity=0.2,
                   max_val=2,
                   radius=7, blur=12, 
                   max_zoom=1, 
                 )
hmap = folium.Map(location=[0,0],zoom_start=2)
hmap.add_child(hm_wide)

Location seems to mostly correlate with densly populated areas. However, there may be some differences in amounts of tweets send, e.g in Bangladesh, India, Southamerica, eastern Europe.